# FTRL



In [1]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt
from random import random
import pickle
import os

In [2]:
os.chdir('/Users/Dam/Desktop')

In [3]:
# A, paths
train='train_sample_df_2.csv'
test='test_sample_df_2.csv'
submission = 'ftrl1sub.csv'  # path of to be outputted submission file

In [11]:
# B, model
alpha = .05  # learning rate
beta = 1.   # smoothing parameter for adaptive learning rate
L1 = .05     # L1 regularization, larger value means more regularized
L2 = 0.     # L2 regularization, larger value means more regularized

In [12]:
# C, feature/hash trick

D = 2 ** 24             # number of weights to use
interaction = False     # whether to enable poly2 feature interactions

# D, training/validation
epoch = 1       # learn training data for N passes
holdafter = None   # data after date N (exclusive) are used as validation
holdout = None  # use every N training instance for holdout validation

In [13]:
##############################################################################
# class, function, generator definitions #####################################
##############################################################################

class ftrl_proximal(object):
    ''' Our main algorithm: Follow the regularized leader - proximal

        In short,
        this is an adaptive-learning-rate sparse logistic-regression with
        efficient L1-L2-regularization

        Reference:
        http://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf
    '''

    def __init__(self, alpha, beta, L1, L2, D, interaction):
        # parameters
        self.alpha = alpha
        self.beta = beta
        self.L1 = L1
        self.L2 = L2

        # feature related parameters
        self.D = D
        self.interaction = interaction

        # model
        # n: squared sum of past gradients
        # z: weights
        # w: lazy weights
        self.n = [0.] * D
        self.z = [random() for k in range(D)]
        self.w = {}

    def _indices(self, x):
        ''' A helper generator that yields the indices in x

            The purpose of this generator is to make the following
            code a bit cleaner when doing feature interaction.
        '''

        # first yield index of the bias term
        yield 0

        # then yield the normal indices
        for index in x:
            yield index

    def predict(self, x):
        ''' Get probability estimation on x

            INPUT:
                x: features

            OUTPUT:
                probability of p(y = 1 | x; w)
        '''

        # parameters
        alpha = self.alpha
        beta = self.beta
        L1 = self.L1
        L2 = self.L2

        # model
        n = self.n
        z = self.z
        w = {}

        # wTx is the inner product of w and x
        wTx = 0.
        for i in self._indices(x):
            sign = -1. if z[i] < 0 else 1.  # get sign of z[i]

            # build w on the fly using z and n, hence the name - lazy weights
            # we are doing this at prediction instead of update time is because
            # this allows us for not storing the complete w
            if sign * z[i] <= L1:
                # w[i] vanishes due to L1 regularization
                w[i] = 0.
            else:
                # apply prediction time L1, L2 regularization to z and get w
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)

            wTx += w[i]

        # cache the current w for update stage
        self.w = w

        # bounded sigmoid function, this is the probability estimation
        return 1. / (1. + exp(-max(min(wTx, 35.), -35.)))

    def update(self, x, p, y):
        ''' Update model using x, p, y

            INPUT:
                x: feature, a list of indices
                p: click probability prediction of our model
                y: answer

            MODIFIES:
                self.n: increase by squared gradient
                self.z: weights
        '''

        # parameter
        alpha = self.alpha

        # model
        n = self.n
        z = self.z
        w = self.w

        # gradient under logloss
        g = p - y

        # update z and n
        for i in self._indices(x):
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g

In [14]:
def logloss(p, y):
    ''' FUNCTION: Bounded logloss

        INPUT:
            p: our prediction
            y: real answer

        OUTPUT:
            logarithmic loss of p given y
    '''

    p = max(min(p, 1. - 10e-15), 10e-15)
    return -log(p) if y == 1. else -log(1. - p)

In [15]:
def data(path, D):
    ''' GENERATOR: Apply hash-trick to the original csv row
                   and for simplicity, we one-hot-encode everything

        INPUT:
            path: path to training or testing file
            D: the max index that we can hash to

        YIELDS:
            ID: id of the instance, mainly useless
            x: a list of hashed and one-hot-encoded 'indices'
               we only need the index since all values are either 0 or 1
            y: y = 1 if we have a click, else we have y = 0
    '''

    for t, row in enumerate(DictReader(open(path), delimiter=',')):
        # process id
        #print row
        
        try:
            display_id = row['display_id']
            ad_id = row['ad_id']
            timestamp_event = row['timestamp_event']
            is_leak = row['is_leak']
            
            del row['display_id']
            del row['ad_id']
            del row['timestamp_event']
            del row['is_leak']
            
        except:
            pass
        
        # process clicks
        y = 0.
        target='label'#'IsClick' 
        if target in row:
            if row[target] == '1':
                y = 1.
            del row[target]

        # build x
        x = []
        for key in row:
            value = row[key]

            # one-hot encode everything with hash trick
            index = abs(hash(key + '_' + value)) % D
            x.append(index)
        
        
        yield t, display_id, ad_id,  x, y

In [16]:
##############################################################################
# start training #############################################################
##############################################################################

start = datetime.now()

# initialize ourselves a learner
learner = ftrl_proximal(alpha, beta, L1, L2, D, interaction)

# start training
for e in range(epoch):
    loss = 0.
    count = 0
    for t, display_id, ad_id, x, y in data(train, D):  # data is a generator

        p = learner.predict(x)
        loss += logloss(p, y)
        learner.update(x, p, y)
        count+=1
        if count%1000==0:
            #print count,loss/unt
            print('%s\tencountered: %d\tcurrent logloss: %f' % (
                datetime.now(), count, loss/count)) 
        if count > 4000:
            break

count=0
loss=0

2018-12-10 23:34:04.683906	encountered: 1000	current logloss: 0.379596
2018-12-10 23:34:04.761407	encountered: 2000	current logloss: 0.390724
2018-12-10 23:34:04.845231	encountered: 3000	current logloss: 0.436090
2018-12-10 23:34:04.924200	encountered: 4000	current logloss: 0.438798


In [33]:
with open(submission, 'w') as outfile:
    outfile.write('display_id,ad_id,label\n')
    for t, display_id, ad_id, x, y in data(test, D):
        p = learner.predict(x)
        outfile.write('%s,%s,%s\n' % (display_id, ad_id, str(p)))

### MAP(Mean Average Precision) 계산

In [ ]:
def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

#    if type(actual) is list:
#        return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])
#    else:
#        apk_array = np.zeros(len(actual))
#        for i in range(len(actual)):
#            apk_array[i] = apk(actual[i], predicted[i], k)
#        return np.mean(apk_array)

## Validation을 통한 Hyper Parameter Tuning